In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
np.random.seed(1)
tensorflow.random.set_seed(1)
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, f1_score

## Data input and management

In [45]:
DB_data = pd.read_excel("Dry_Bean_Dataset.xlsx")

In [46]:
x_total = DB_data.drop(['Class'], axis=1) # everything except the class # DB for Dry beans
y = DB_data['Class']  # only the class


In [47]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Reshape the input vector to a 2D array with a single column
input_array = [[item] for item in y]

# Fit and transform the input array using the encoder
y_total = encoder.fit_transform(input_array)

In [48]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_total, y_total, test_size=0.1, random_state=1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)

#scale
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [486]:
y.unique()

array(['SEKER', 'BARBUNYA', 'BOMBAY', 'CALI', 'HOROZ', 'SIRA', 'DERMASON'],
      dtype=object)

In [487]:
y_train

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [488]:
y_test

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [489]:
y_val

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [490]:
x_train

array([[-0.45230498, -0.56642518, -0.83998333, ...,  1.49283775,
         1.80624901,  0.85834145],
       [-0.31036032, -0.29539358, -0.14142003, ..., -0.35725861,
        -0.54563597,  0.14303312],
       [ 2.986967  ,  2.66320004,  2.3909027 , ..., -1.24784071,
         0.07350478, -0.23572825],
       ...,
       [ 0.72137049,  0.87385458,  1.01284472, ..., -1.03144271,
        -0.73141171, -0.97885949],
       [-0.16323619, -0.06995348, -0.07942365, ..., -0.24691538,
        -0.17826263, -0.04219919],
       [ 0.66856017,  0.91924172,  1.0724145 , ..., -1.13677331,
        -0.99127158, -1.96737601]])

## NN from library

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import make_scorer, f1_score
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils


In [63]:
# Define the function to create the model
def create_model(units = 200):
    model = Sequential()
    model.add(Dense(units, activation='tanh', input_dim=len(x_train[0])))
    model.add(Dense(units, activation='tanh'))
    model.add(Dense(units=len(y_train[0]), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
#f1_score(y_test, y_pred)
cr = cross_validate(model, x_train, y_train, cv=5)

C:\Users\38599\AppData\Local\Temp/ipykernel_17116/2427224783.py:11: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


69/69 [==============================] - 0s 2ms/step - loss: 0.2358 - accuracy: 0.9165


In [64]:
cr['test_score'].mean()

0.9155479788780212